## LightFM
[Metadata Embeddings for User and Item Cold-start Recommendations](https://arxiv.org/pdf/1507.08439.pdf)

这篇文章主要是为了兼顾MF(Matrix Factorization)和CB(Content-Based)。MF需要warm data（即需要user-item interaction），CB却可以一定程度上解决cold-start，所以融合两者的好处就是避免了训练两个模型。因为CB一般是要比MF效果更差，所以不可能只使用CB。
为了兼顾cold-start和warm data的良好效果，Lyst提出了LightFM方法，同时拥有协同过滤和CB的优点。LightFM有以下特点：
1. 在cold-start和low density场景下，LightFM不会比CB效果更差，如果colaborative信息可用，则一般会比CB更好
2. 当collaborative data很多（warm-start），则LightFM不会比MF差
3. LightFM包括特征重要的语义信息，这些可以用于诸如标签推荐等相关性推荐任务。

另外，由于可以使用user和Item的metadata，所以可以同时应对user cold-start和item cold-start的场景。

### 介绍

有两个任务需要完成：
1. 模型必须能够通过interaction data学习user和item的表现，例如：users都同时喜欢itemA和itemB，则说明itemA和itemB相似
2. 模型必须能够对new item和new user计算推荐结果

为了完成任务1，模型使用隐含表示方法（latent representation approach）；为了完成任务2，将items和users表示成他们content features的线性组合。

#### 模型

假设$U$是users集，$I$是items集，$F^U$是user的特征集，$F^I$是item的特征集。每一个user会对一些item产生交互（例如评分）。假如是一种二值评价方法（正面或者负面），所有的user-item interaction数据集是$(u,i)\in U\times I$，这个集合可以分成正负评价$S^+$和$S^-$。

每一个user和每一个item都被一组特征描述，分别表示为$f_u\subset F^U$, $f_i\subset F^I$。

对于每一个特征，根据d维user/item特征嵌入$e_f^U$和$e_f^I$来对模型进行参数化，并且每一特特征还有一个常量的bias分别表示为$b_f^U,b_f^I$

对于user u的隐含表示，通过它的特征的隐含向量求和获得，bias也一样：
$$q_u = \sum_{j\in f_u}e_j^U, b_u = \sum_{j\in f_u}e_j^U$$
对于item同上：
$$p_i = \sum_{j\in f_i}e_j^I, b_i = \sum_{j\in f_i}e_j^I$$

模型预测user u对item i的评分如下所示：
$$\hat r_{ui} = f(q_u\cdot p_i + b_u + b_i)$$
这里的$f$可以根据需要选择，因为根据上面的假设是预测二分类数据，所以选择sigmoid函数：
$$f(x) = \frac{1}{1 + exp(-x)}$$

使用最大化似然来求参数值，似然函数如下：
$$
L(e^U,e^I,b^U,b^I)=\prod_{(u,i)\in S^+}\hat r_{ui}\times\prod_{(u,i)\in S^-}(1 - \hat r_{ui})
$$

如果feature set完全由每一个user和item的标识位组成，则LightFM就和MF类似。如果feature set中也包含了item或user所共有的metadata features，LightFM则扩展MF模型，让feature latent factors解释为user interaction结构的一部分。

1. 在大多数应用中，相比user和item，metadata features的数量会更少，就会意味着更少的参数，从而降低过拟合的风险和提升泛化能力以及运行效率
2. 指示变量的隐含向量不能有效处理cool-start，融合metadata features就会有效处理
3. 如果仅仅只有标识特征（indicator features）可用，则LightFM和标准的MF模型类似。

如果仅仅只有metadata feature，而没有标识特征，LightFM并不和CB模型等同。但是有一个特例，如果each user通过一个标识变量来描述（这个标识变量只和一个item相关），则user Vector在LSI模型中等价于一个document vector，

In [ ]:
#read the code and answer the question

问题：
* latent feature指的是metadata features还是user-item interaction？
* 具体的算法的参数指的是什么？
* 数据的表现形式是什么？
* 数据在模型中的作用如何体现？
* indicator feature到底是不是user-item interaction？
* embeddings的具体含义？所指为何物？
* LightFM等价于MF和CB的两个例子下是如何等价的？

### 补充说明
#### LSI
LSI：latent semantic index

#### Content-Based


#### Matrix Factorization
MF在CF中扮演一个重要的角色，因为随着user-item rating的增加，CF计算会耗费很多计算资源，或许几百万数据就已经难以按照CF的算法计算。这时候使用MF来计算可能就有很大的好处了。

MF主要作为一种无监督学习方法来对隐含变量进行分级和降维，评分预测以及推荐可以通过很多算法获得，但是基于最近邻（Neighborhood-based）方法是简单且直觉有效的。MF技术通常更有效，因为他可以在user-item interaction中发现隐含的特征（features）。MF是一个数学工具，被应用在很多应用中，其实就是对矩阵的一些操作，用来发现隐藏在数据中的信息。

SVD和PCA是众所周知的两种MF模型，用来在IR领域内获取隐含因子，从而来解决CF的计算困境。